In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**# This Data_Loader file **

In [0]:
import numpy as np
# count=0
# for index, line in enumerate(open('tokyo.txt','r')):
#     count += 1
# print (count)# = 573703

positive_examples = list(open('/content/gdrive/My Drive/tokyo.txt', "r").readlines())
positive_examples = [s for s in positive_examples]
items = np.array(list((positive_examples)))
#print(item)

**Interreaction**

In [0]:
"""
Classes describing datasets of user-item interactions. Instances of these
are returned by dataset-fetching and dataset-processing functions.
"""

import numpy as np

import scipy.sparse as sp


class Interactions(object):
    """
    Interactions object. Contains (at a minimum) pair of user-item
    interactions. This is designed only for implicit feedback scenarios.

    Parameters
    ----------

    file_path: file contains (user,item,rating) triplets
    user_map: dict of user mapping
    item_map: dict of item mapping
    """

    def __init__(self,data,
                 user_map=None,
                 item_map=None):

        if not user_map and not item_map:
            user_map = dict()
            item_map = dict()

            num_user = 0
            num_item = 0
        else:
            num_user = len(user_map)
            num_item = len(item_map)

        user_ids = list()
        item_ids = list()
        # read users and items from file
        for item in data:
            u, i, latitude, longitude, time = item.split(',')
            user_ids.append(u)
            item_ids.append(i)
        print(user_ids)
        

        # update user and item mapping
        for u in user_ids:
            if u not in user_map:
                user_map[u] = num_user
                num_user += 1
        for i in item_ids:
            if i not in item_map:
                item_map[i] = num_item
                num_item += 1

        user_ids = np.array([user_map[u] for u in user_ids])
        item_ids = np.array([item_map[i] for i in item_ids])

        self.num_users = num_user
        self.num_items = num_item

        self.user_ids = user_ids
        self.item_ids = item_ids

        self.user_map = user_map
        self.item_map = item_map

        self.sequences = None
        self.test_sequences = None

    def __len__(self):

        return len(self.user_ids)

    def tocoo(self):
        """
        Transform to a scipy.sparse COO matrix.
        """

        row = self.user_ids
        col = self.item_ids
        data = np.ones(len(self))

        return sp.coo_matrix((data, (row, col)),
                             shape=(self.num_users, self.num_items))

    def tocsr(self):
        """
        Transform to a scipy.sparse CSR matrix.
        """

        return self.tocoo().tocsr()

    def to_sequence(self, sequence_length=5, target_length=1):
        """
        Transform to sequence form.

        Valid subsequences of users' interactions are returned. For
        example, if a user interacted with items [1, 2, 3, 4, 5, 6, 7, 8, 9], the
        returned interactions matrix at sequence length 5 and target length 3
        will be be given by:

        sequences:

           [[1, 2, 3, 4, 5],
            [2, 3, 4, 5, 6],
            [3, 4, 5, 6, 7]]

        targets:

           [[6, 7],
            [7, 8],
            [8, 9]]

        sequence for test (the last 'sequence_length' items of each user's sequence):

        [[5, 6, 7, 8, 9]]

        Parameters
        ----------

        sequence_length: int
            Sequence length. Subsequences shorter than this
            will be left-padded with zeros.
        target_length: int
            Sequence target length.
        """

        # change the item index start from 1 as 0 is used for padding in sequences
        for k, v in self.item_map.items():
            self.item_map[k] = v + 1
        self.item_ids = self.item_ids + 1
        self.num_items += 1

        max_sequence_length = sequence_length + target_length #5+3/1

        # Sort first by user id
        sort_indices = np.lexsort((self.user_ids,))

        user_ids = self.user_ids[sort_indices]
        item_ids = self.item_ids[sort_indices]

        user_ids, indices, counts = np.unique(user_ids,
                                              return_index=True,
                                              return_counts=True)

        num_subsequences = sum([c - max_sequence_length + 1 if c >= max_sequence_length else 1 for c in counts])

        sequences = np.zeros((num_subsequences, sequence_length),
                             dtype=np.int64)
        sequences_targets = np.zeros((num_subsequences, target_length),
                                     dtype=np.int64)
        sequence_users = np.empty(num_subsequences,
                                  dtype=np.int64)

        test_sequences = np.zeros((self.num_users, sequence_length),
                                  dtype=np.int64)
        test_users = np.empty(self.num_users,
                              dtype=np.int64)

        _uid = None
        for i, (uid,
                item_seq) in enumerate(_generate_sequences(user_ids,
                                                           item_ids,
                                                           indices,
                                                           max_sequence_length)):
            if uid != _uid:
                test_sequences[uid][:] = item_seq[-sequence_length:]
                test_users[uid] = uid
                _uid = uid
            sequences_targets[i][:] = item_seq[-target_length:]
            sequences[i][:] = item_seq[:sequence_length]
            sequence_users[i] = uid

        self.sequences = SequenceInteractions(sequence_users, sequences, sequences_targets)
        self.test_sequences = SequenceInteractions(test_users, test_sequences)


class SequenceInteractions(object):
    """
    Interactions encoded as a sequence matrix.

    Parameters
    ----------
    user_ids: np.array
        sequence users
    sequences: np.array
        The interactions sequence matrix, as produced by
        :func:`~Interactions.to_sequence`
    targets: np.array
        sequence targets
    """

    def __init__(self,
                 user_ids,
                 sequences,
                 targets=None):
        self.user_ids = user_ids
        self.sequences = sequences
        self.targets = targets

        self.L = sequences.shape[1]
        self.T = None
        if np.any(targets):
            self.T = targets.shape[1]


def _sliding_window(tensor, window_size, step_size=1):
    if len(tensor) - window_size >= 0:
        for i in range(len(tensor), 0, -step_size):
            if i - window_size >= 0:
                yield tensor[i - window_size:i]
            else:
                break
    else:
        num_paddings = window_size - len(tensor)
        # Pad sequence with 0s if it is shorter than windows size.
        yield np.pad(tensor, (num_paddings, 0), 'constant')


def _generate_sequences(user_ids, item_ids,
                        indices,
                        max_sequence_length):
    for i in range(len(indices)):

        start_idx = indices[i]

        if i >= len(indices) - 1:
            stop_idx = None
        else:
            stop_idx = indices[i + 1]

        for seq in _sliding_window(item_ids[start_idx:stop_idx],
                                   max_sequence_length):
            yield (user_ids[i], seq)


**utils**

In [0]:
import numpy as np

def sample_top(a=[], top_k=10):
    idx = np.argsort(a)[::-1]
    idx = idx[:top_k]
    probs = a[idx]
    probs = probs / np.sum(probs)
    choice = np.random.choice(idx, p=probs)
    return choice

# fajie
def sample_top_k(a=[], top_k=10):
    idx = np.argsort(a)[::-1]
    idx = idx[:top_k]
    # probs = a[idx]
    # probs = probs / np.sum(probs)
    # choice = np.random.choice(idx, p=probs)
    return idx

print sample_top_k(np.array([0.02,0.01,0.01,0.16,0.8]),3)




[4 3 0]


**text_cnn_hv.py**

In [0]:
import tensorflow as tf
import numpy as np

'''
including verical and horizonal cnn
'''
class TextCNN_hv(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    "https://github.com/dennybritz/cnn-text-classification-tf"
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, loss_type, l2_reg_lambda):

        # Placeholders for input, output and dropout



        self.source_sess = tf.placeholder('int32',
                                         [None, None], name='source_sess')
        self.target_sess = tf.placeholder('int32',
                                         [None, None], name='target_sess')

        
#         text_batch.sequences.sequences
#         text_batch.sequences.targets,
#         text_batch.sequences.user_ids.reshape(-1, 1)

        
        
        
        
        
        
#         source_sess = self.wholesession[:, 0:-1]
#         target_sess = self.wholesession[:, -1:]

        new_sequence_length=4

        self.input_x=self.source_sess
        self.input_y=self.target_sess
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        self.loss_type = loss_type
        self.l2_reg_lambda = l2_reg_lambda


        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)
        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

  
        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                # http://www.infoq.com/cn/articles/introduction-of-tensorflow-part4   how to use cnn
                # new shape after conv2d[?, new_sequence_length - filter_size + 1, 1, 1]
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                # new shape after max_pool[?, 1, 1, num_filters]
                # be carefyul, the  new_sequence_length has changed because of wholesession[:, 0:-1]
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, new_sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total]) #shape=[batch_size, 384]
        # design the veritcal cnn
        with tf.name_scope("conv-verical" ):
            filter_shape = [new_sequence_length, 1, 1, 1]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[1]), name="b")
            conv = tf.nn.conv2d(
                self.embedded_chars_expanded,
                W,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
        self.vcnn_flat= tf.reshape(h, [-1, embedding_size])
        self.final=tf.concat([self.h_pool_flat,self.vcnn_flat],1) #shape=[batch_size, 384+100]


        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.final, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total+embedding_size, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            # self.predictions = tf.argmax(self.scores, 1, name="predictions")
            # losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.input_y = tf.reshape(self.input_y, [-1])
            self.loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.input_y, logits=self.scores)
            self.loss = tf.reduce_mean(self.loss + l2_reg_lambda * l2_loss)

            self.probs_flat = tf.nn.softmax(self.scores)
            self.arg_max_prediction = tf.argmax(self.probs_flat, 1)

**参数类**

In [0]:
class Args(object):
    """docstring for Hotel"""
    def __init__(self,
                 learning_rate = 0.003,
                 batch_size=512, 
                 sample_every=4000,
                 summary_every=50,
                 save_model_every = 1500,
                 sample_size=300,
                 top_k = 5,
                 max_epochs = 50,
                 beta1 = 0.5,
                 resume_model = None,
                 text_dir ='/content/gdrive/My Drive/tokyo.txt' ,
                 data_dir = '/content/gdrive/My Drive/',
                 seed='f78c95a8-9256-4757-9a9f-213df5c6854e,1151b040-8022-4965-96d2-8a4605ce456c',
                 sample_percentage = 0.2,
                 filter_sizes = '[2,3,4]',
                 num_filters = 100,#128
                 loss_type = 'square_loss',#'Specify a loss type (square_loss or log_loss).'
                 l2_reg_lambda = 0,
                 allow_soft_placement = True,
                 log_device_placement = False,
                 dropout_keep_prob =0.5
                 
                ):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.sample_every = sample_every
        self.summary_every = summary_every
        self.save_model_every = save_model_every
        self.sample_size = sample_size
        self.top_k = top_k
        self.max_epochs = max_epochs
        self.beta1 =beta1
        self.resume_model = resume_model
        self.text_dir=text_dir
        self.data_dir=data_dir
        self.seed=seed
        self.sample_percentage=sample_percentage
        self.filter_sizes=filter_sizes
        self.num_filters=num_filters
        self.loss_type=loss_type
        self.l2_reg_lambda=l2_reg_lambda
        self.allow_soft_placement=allow_soft_placement
        self.log_device_placement=log_device_placement
        self.dropout_keep_prob=dropout_keep_prob


**caser**

In [6]:
import tensorflow as tf
import numpy as np
import argparse
import shutil
import time
import os
import sys
import math


'''
reimplementation of
Personalized Top-N Sequential Recommendation via
Convolutional Sequence Embedding
screen print has been changed a bit so that to print the output not that ofen
'''


def main():
   
    args = Args()



    all_samples = items
    


    # Randomly shuffle data
    np.random.seed(10)
   
    shuffle_indices = np.random.permutation(np.arange(len(all_samples)))
    text_samples = all_samples[shuffle_indices]



    # Split train/test set
    # TODO: This is very crude, should use cross-validation
    dev_sample_index = -1 * int(args.sample_percentage * float(len(text_samples)))

    count = 573703
#     for index, line in enumerate(open('/content/gdrive/My Drive/tokyo.txt','r')):
#         count += 1
#     print count
    dev_sample_index = -1 * int(args.sample_percentage * float(count))
    x_train, x_dev = text_samples[:dev_sample_index], text_samples[dev_sample_index:]
    print ('x_train.shape[0]')
    print (x_train.shape[0])
    print ('x_dev.shape[0]')
    print (x_dev.shape[0])
#     

#     #create subsession only for training
#     subseqtrain = []
#     for i in range(len(x_train)):
#         #print x_train[i]
#         seq=x_train[i]
#         lenseq=len(seq)
#         #session lens=100 shortest subsession=5 realvalue+95 0
#         for j in range(lenseq-4):
#             subseqend=seq[:len(seq)- j]
#             subseqbeg=[0]*j
#             subseq= np.append(subseqbeg,subseqend)
#             #beginseq=padzero+subseq
#             #newsubseq=pad+subseq
#             subseqtrain.append(subseq)
#     x_train=np.array(subseqtrain) #list to ndarray
#     del subseqtrain
#     # Randomly shuffle data
#     np.random.seed(10)
#     shuffle_train = np.random.permutation(np.arange(len(x_train)))
#     x_train = x_train[shuffle_train]
#     print "generating subsessions is done!"
#     print "shape", x_train.shape[0]
#     print "dataset",args.text_dir
    model_options = {
        'vocab_size': len(items),
        'residual_channels':50,
    }


    cnn = TextCNN_hv(
        sequence_length=8,
        num_classes=61858,
        vocab_size=61858,
        embedding_size= model_options['residual_channels'],
        filter_sizes=eval(args.filter_sizes),
        num_filters=args.num_filters,
        loss_type=args.loss_type,
        l2_reg_lambda=args.l2_reg_lambda)

    print "embedding_size", model_options['residual_channels']
    session_conf = tf.ConfigProto(
        # allow to distribute device automatically if your assigned device is not found
        allow_soft_placement=args.allow_soft_placement,
        # whether print or not
        log_device_placement=args.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        sess.run(tf.global_variables_initializer())



    text_batch = Interactions(x_train)
    text_batch.to_sequence(4, 1)
    sequences = text_batch.sequences.sequences
    targets = text_batch.sequences.targets
    
    test_text_batch = Interactions(x_dev)
    test_text_batch.to_sequence(4, 1)
    test_sequences = text_batch.sequences.sequences
    test_targets = text_batch.sequences.targets

    step = 1
    for epoch in range(args.max_epochs):
        batch_no = 0
        batch_size = args.batch_size
        while (batch_no + 1) * batch_size < x_train.shape[0]:
#             text_batch = text_batch[batch_no * batch_size: (batch_no + 1) * batch_size]
#                     # a=self.t_sentence.get_shape()[1]*2
#             print(text_batch)
            
            source_sess = sequences[batch_no * batch_size: (batch_no + 1) * batch_size]
            target_sess = targets[batch_no * batch_size: (batch_no + 1) * batch_size]

            #users = text_batch.sequences.user_ids.reshape(-1, 1)
            start = time.clock()
            _, loss, prediction = sess.run(
                [train_op, cnn.loss,
                 cnn.arg_max_prediction],
                feed_dict={
                    cnn.source_sess: source_sess,
                    cnn.target_sess:target_sess,
                    cnn.dropout_keep_prob: args.dropout_keep_prob
                })
            end = time.clock()
            if step % args.sample_every == 0:
                print "-------------------------------------------------------train1"
                print "LOSS: {}\tEPOCH: {}\tBATCH_NO: {}\t STEP:{}\t total_batches:{}".format(
                    loss, epoch, batch_no, step, x_train.shape[0] / args.batch_size)
                print "TIME FOR BATCH", end - start
                print "TIME FOR EPOCH (mins)", (end - start) * (x_train.shape[0] / args.batch_size) / 60.0

            if step % args.sample_every == 0:
                print "-------------------------------------------------------test1"
                if (batch_no + 1) * batch_size < x_dev.shape[0]:
                    #text_batch = x_dev[(batch_no) * batch_size: (batch_no + 1) * batch_size]
                     
                    source_sess = test_sequences[batch_no * batch_size: (batch_no + 1) * batch_size]
                    target_sess = test_targets[batch_no * batch_size: (batch_no + 1) * batch_size]
                loss = sess.run(
                    [cnn.loss],
                    feed_dict={
                        cnn.source_sess: source_sess,
                        cnn.target_sess:target_sess,
                        cnn.dropout_keep_prob: 1.0
                    })
                print "LOSS: {}\tEPOCH: {}\tBATCH_NO: {}\t STEP:{}\t total_batches:{}".format(
                    loss, epoch, batch_no, step, x_dev.shape[0] / args.batch_size)



            batch_no += 1

            if step % args.sample_every == 0:
                print "********************************************************accuracy"
                batch_no_test = 0
                batch_size_test = batch_size*2
                curr_preds_5 = []
                rec_preds_5 = []  # 1
                ndcg_preds_5 = []  # 1
                curr_preds_20 = []
                rec_preds_20 = []  # 1
                ndcg_preds_20 = []  # 1
                while (batch_no_test + 1) * batch_size_test < x_dev.shape[0]:
                    print(batch_no_test)
                    print(batch_size_test)
                    print(x_dev.shape[0])
                    print(step)
                    print(step / (args.sample_every))
                    # do not need to evaluate all, only after several 10 sample_every, then output final results
                    if(step / (args.sample_every)<10):
                        if (batch_no_test > 2):
                            break
                    else:
                        if (batch_no_test > 500):
                            break
                    #text_batch = x_dev[batch_no_test * batch_size_test: (batch_no_test + 1) * batch_size_test, :]
                    test_source_sess = test_sequences[batch_no_test * batch_size_test: (batch_no_test + 1) * batch_size_test]
                    test_target_sess = test_targets[batch_no_test * batch_size_test: (batch_no_test + 1) * batch_size_test]
                    [probs] = sess.run(
                        [cnn.probs_flat],
                        feed_dict={
                            #cnn.wholesession: text_batch,
                            cnn.source_sess: test_source_sess,
                            cnn.target_sess:test_target_sess,
                            cnn.dropout_keep_prob: 0.5
                        })
                    for bi in range(probs.shape[0]):
                        pred_words_5 = sample_top_k(probs[bi], top_k=args.top_k)  # top_k=5
                        pred_words_20 = sample_top_k(probs[bi], top_k=args.top_k + 15)

                        true_word = int(test_target_sess[bi])
                        predictmap_5 = {ch: i for i, ch in enumerate(pred_words_5)}
                        pred_words_20 = {ch: i for i, ch in enumerate(pred_words_20)}

                        rank_5 = predictmap_5.get(true_word)
                        rank_20 = pred_words_20.get(true_word)
                        if rank_5 == None:
                            curr_preds_5.append(0.0)
                            rec_preds_5.append(0.0)  # 2
                            ndcg_preds_5.append(0.0)  # 2
                        else:
                            MRR_5 = 1.0 / (rank_5 + 1)
                            Rec_5 = 1.0  # 3
                            ndcg_5 = 1.0 / math.log(rank_5 + 2, 2)  # 3
                            curr_preds_5.append(MRR_5)
                            rec_preds_5.append(Rec_5)  # 4
                            ndcg_preds_5.append(ndcg_5)  # 4
                        if rank_20 == None:
                            curr_preds_20.append(0.0)
                            rec_preds_20.append(0.0)  # 2
                            ndcg_preds_20.append(0.0)  # 2
                        else:
                            MRR_20 = 1.0 / (rank_20 + 1)
                            Rec_20 = 1.0  # 3
                            ndcg_20 = 1.0 / math.log(rank_20 + 2, 2)  # 3
                            curr_preds_20.append(MRR_20)
                            rec_preds_20.append(Rec_20)  # 4
                            ndcg_preds_20.append(ndcg_20)  # 4

                    batch_no_test += 1
                    print "BATCH_NO: {}".format(batch_no_test)
                    print "Accuracy mrr_5:", sum(curr_preds_5) / float(len(curr_preds_5))  # 5
                    print "Accuracy mrr_20:", sum(curr_preds_20) / float(len(curr_preds_20))  # 5
                    print "Accuracy hit_5:", sum(rec_preds_5) / float(len(rec_preds_5))  # 5
                    print "Accuracy hit_20:", sum(rec_preds_20) / float(len(rec_preds_20))  # 5
                    print "Accuracy ndcg_5:", sum(ndcg_preds_5) / float(len(ndcg_preds_5))  # 5
                    print "Accuracy ndcg_20:", sum(ndcg_preds_20) / float(len(ndcg_preds_20))  #
                    # print "curr_preds",curr_preds

            step += 1




if __name__ == '__main__':
    main()

print('DONENNNNNNNeeeeeeeeeee')



x_train.shape[0]
458963
x_dev.shape[0]
114740
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

embedding_size 50
Instructions for updating:
Use tf.cast instead.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['1426', '659', '69', '127', '2088', '952', '922', '151', '1186', '570', '2232', '2171', '1172', '1143', '1912', '1745', '1216', '1018', '1381', '1612', '1892', '1491', '1312', '1894', '1516', '1290', '1290', '1868', '1851', '431', '232', '670', '1291', '291', '279', '300', '414', '2191', '1439', '557', '2045', '444', '135', '1493', '932', '1300', '1689', '1158', '822', '1307', '872', '1909', '690', '2151', '2245', '73', '1651', '1603', '472', '1511', '35', '1240', '780', '493', '616', '1343', '1961', '824', '1282', '740', '1791', '863', '1188', '1413', '366', '1336', '856', '2013', '1873', '2156', '1157', '1693', '197', '594', '1584', '2026', '1724', '459', '1777', '65', '910', '1744', '485', '100', '104', '348', '162', '1401', '1039', '779', '1550', '1985', '354', '1108', '780', '1', '94', '536', '275', '1561', '1431', '397', '910', '77', '255', '397', '884', '1645', '1952', '271', '1532', '442', '2097', '323', '9', '2063', '1855', '251', '393', '1448', '822', '604', '1896', '15', '1